In [54]:
import pandas as pd
import os

In [55]:
sumZoneTripAndMode =  "E:\GitHub\WF-TDM-v9x\_working-bill\results"
scnJson = "_site/scenarios.json"
outputJsonName = "zones-modeshare-vizmap.json"

In [56]:
# attributes for road seg summaries
dfAttributes = pd.DataFrame([
    ["aT"   , "Number of Trips"               ],
#   ["aTrLen", "Average Auto Trip Length"     ],
#   ["aTrLen", "Average Transit Trip Distance"],
#   ["aVmtTr", "Trip VMT"                     ],
    ["aAll", "Share of All Trips"             ],
    ["aMot", "Share of Motorized Trips"       ],
    ["aNon", "Share of Non-Motorized Trips"   ],
    ["aAut", "Share of Auto Trips"            ],
    ["aTrn", "Share of Transit Trips"         ],
    ["aWlk", "Share of Walk Trips"            ],
    ["aBik", "Share of Bike Trips"            ],
    ["aLcl", "Share of Local Bus"             ],
    ["aCor", "Share of Core Bus"              ],
    ["aExp", "Share of Express Bus"           ],
    ["aLrt", "Share of Light-Rail Transit"    ],
    ["aCrt", "Share of Commuter-Rail Transit" ],
    ["aBrt", "Share of Bus-Rapid Transit"     ]
], columns=('aCode','aName'))
display(dfAttributes)

dfFilters = pd.DataFrame([
    ["fPurp","Trip Purpose"         , "select", True ],
    ["fMode","Mode"                 , "select", True ],
    ["fTod" ,"Time of Day"          , "select", True ],
    ["fPA"  ,"Production/Attraction", "select", False]
], columns=('fCode','fName','fWidget','fOwnCol'))
display(dfFilters)

lstFiltersOwnCol = dfFilters[dfFilters['fOwnCol'] == True]['fCode'].tolist()
display(lstFiltersOwnCol)


,aCode,aName
0,aT,Number of Trips
1,aSAll,Share of All Trips
2,aSMot,Share of Motorized Trips
3,aSNon,Share of Non-Motorized Trips
4,aSAut,Share of Auto Trips
5,aSTrn,Share of Transit Trips
6,aSWlk,Share of Walk Trips
7,aSBik,Share of Bike Trips
8,aSLcl,Share of Local Bus
9,aSCor,Share of Core Bus


,fCode,fName,fWidget,fOwnCol
0,fPurp,Trip Purpose,select,True
1,fMode,Mode,select,True
2,fTod,Time of Day,select,True
3,fPA,Production/Attraction,select,False


['fPurp', 'fMode', 'fTod']

In [57]:
# MAKE SURE ONLY COLUMNS WITH LOWER-CASE "f" ARE FILTER COLUMNS AND LOWER-CASE "a" ARE ATTRIBUTE COLUMNS

dfColKeys = pd.DataFrame([
    ["aProd"     , 'P', 'aT'],
    ["aAttr"     , 'A', 'aT'],
    ['aShAllProd', 'P', 'aAll'],
    ['aShAllAttr', 'A', 'aAll'],
    ['aShMotProd', 'P', 'aMot'],
    ['aShMotAttr', 'A', 'aMot'],
    ['aShNonProd', 'P', 'aNon'],
    ['aShNonAttr', 'A', 'aNon'],
    ['aShAutProd', 'P', 'aAut'],
    ['aShAutAttr', 'A', 'aAut'],
    ['aShTrnProd', 'P', 'aTrn'],
    ['aShTrnAttr', 'A', 'aTrn'],
    ['aShWlkProd', 'P', 'aWlk'],
    ['aShWlkAttr', 'A', 'aWlk'],
    ['aShBikProd', 'P', 'aBik'],
    ['aShBikAttr', 'A', 'aBik'],
    ['aShLclProd', 'P', 'aLcl'],
    ['aShLclAttr', 'A', 'aLcl'],
    ['aShCorProd', 'P', 'aCor'],
    ['aShCorAttr', 'A', 'aCor'],
    ['aShExpProd', 'P', 'aExp'],
    ['aShExpAttr', 'A', 'aExp'],
    ['aShLrtProd', 'P', 'aLrt'],
    ['aShLrtAttr', 'A', 'aLrt'],
    ['aShCrtProd', 'P', 'aCrt'],
    ['aShCrtAttr', 'A', 'aCrt'],
    ['aShBrtProd', 'P', 'aBrt'],
    ['aShBrtAttr', 'A', 'aBrt']
], columns=('col','fPA','aCode'))

dfColKeys

,col,fPA,aCode
0,aProd,P,aT
1,aAttr,A,aT
2,aShAllProd,P,aSAll
3,aShAllAttr,A,aSAll
4,aShMotProd,P,aSMot
5,aShMotAttr,A,aSMot
6,aShNonProd,P,aSNon
7,aShNonAttr,A,aSNon
8,aShAutProd,P,aSAut
9,aShAutAttr,A,aSAut


In [58]:
dfScn = pd.read_json(scnJson)
dfScn = dfScn[['modVersion','scnGroup','scnYear']]
display(dfScn)
display(dfScn.dtypes)

,modVersion,scnGroup,scnYear
0,v900,Base,2019
1,v900,Base,2023
2,v900,TIP,2028
3,v900,RTP,2032
4,v900,RTP,2042
5,v900,RTP,2050
6,v900,NoBuild,2032
7,v900,NoBuild,2042
8,v900,NoBuild,2050
9,v900,Needs,2032


modVersion    object
scnGroup      object
scnYear        int64
dtype: object

In [59]:
# create folders
# Define a root directory where you want to create subfolders
root_dir = './_site/data/scnData'

# Ensure the root directory exists
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

# Loop through the dataframe and create subfolders
for index, row in dfScn.iterrows():
    folder_name = f"{row['modVersion']}__{row['scnGroup']}__{row['scnYear']}"
    full_path = os.path.join(root_dir, folder_name)
    
    if not os.path.exists(full_path):
        os.makedirs(full_path)


In [61]:
import pandas as pd
import os
import glob

# Define the subfolder path
subfolder_path = r'E:\GitHub\WF-TDM-v9x\_working-bill\results'

# Create an empty list to store dataframes
_dfs = []

# Iterate through the scenarios
for _, row in dfScn.iterrows():
    print(row['scnGroup'] + '_' + str(row['scnYear']))
    
    # Read the CSV into a dataframe
    _df = pd.read_csv(os.path.join(subfolder_path, row['scnGroup'] + '_' + str(row['scnYear']) + '_ZoneSummary_TripsAndModeShare.csv'))
    #display(_df)

    _df_melt = _df.melt(id_vars=['TAZID'] + lstFiltersOwnCol, value_vars=dfColKeys['col'].tolist(), var_name='col', value_name='val')
    #display(_df_melt)
    _df_melt_withkeys = pd.DataFrame.merge(_df_melt, dfColKeys, on='col')
    #display(_df_melt_withkeys)

    _df_melt_withkeys_pivot_atts = _df_melt_withkeys.pivot(index=[col for col in _df_melt_withkeys.columns if col not in ['aCode','val','col']], columns='aCode', values="val")
    _df_melt_withkeys_pivot_atts.columns.name = None
    _df_melt_withkeys_pivot_atts.reset_index(inplace=True)
    _df_melt_withkeys_pivot_atts
    #display(_df_melt_withkeys_pivot_atts)

    # Add new columns with values from filename
    _df_melt_withkeys_pivot_atts['modVersion'] = row['modVersion']
    _df_melt_withkeys_pivot_atts['scnGroup'] = row['scnGroup']
    _df_melt_withkeys_pivot_atts['scnYear'] = row['scnYear']

    _df_melt_withkeys_pivot_atts['scnYear'] = _df_melt_withkeys_pivot_atts['scnYear'].astype('int64')

    _df_melt_withkeys_pivot_atts['aT'] = round(_df_melt_withkeys_pivot_atts['aT'],2)
    _df_melt_withkeys_pivot_atts['aT'] = _df_melt_withkeys_pivot_atts['aT'].astype(int)
    _df_melt_withkeys_pivot_atts

    _dfs.append(_df_melt_withkeys_pivot_atts)

# If you want to concatenate all the dataframes into a single dataframe
print('combining dataframes...')
dfCombined_melt_withkeys_pivot_atts = pd.concat(_dfs, ignore_index=True)
display(dfCombined_melt_withkeys_pivot_atts)
print('Done!')


Base_2019
Base_2023
TIP_2028
RTP_2032
RTP_2042
RTP_2050
NoBuild_2032
NoBuild_2042
NoBuild_2050
Needs_2032
Needs_2042
Needs_2050
Needs_2042
Needs MAG_2050
combining dataframes...


,TAZID,fPurp,fMode,fTod,fPA,aSAll,aSAut,aSBik,aSBrt,aSCor,...,aSLcl,aSLrt,aSMot,aSNon,aSTrn,aSWlk,aT,modVersion,scnGroup,scnYear
0,2,AllPurp,All,Dy,A,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,39,v900,Base,2019
1,2,AllPurp,All,Dy,P,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,145,v900,Base,2019
2,2,AllPurp,All,Ok,A,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,26,v900,Base,2019
3,2,AllPurp,All,Ok,P,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,92,v900,Base,2019
4,2,AllPurp,All,Pk,A,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,13,v900,Base,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29008935,3629,HBW,sr3p,Dy,P,0.0797,0.0812,0.0,0.0,0.0,...,0.0,0.0,0.0797,0.0,0.0,0.0,25,v900,Needs MAG,2050
29008936,3629,HBW,sr3p,Ok,A,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0,v900,Needs MAG,2050
29008937,3629,HBW,sr3p,Ok,P,0.0455,0.0459,0.0,0.0,0.0,...,0.0,0.0,0.0455,0.0,0.0,0.0,5,v900,Needs MAG,2050
29008938,3629,HBW,sr3p,Pk,A,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0,v900,Needs MAG,2050


Done!


,TAZID,fPurp,fMode,fTod,fPA,aShAll,aShAut,aShBik,aShBrt,aShCor,...,aShLcl,aShLrt,aShMot,aShNon,aShTrn,aShWlk,aTrips,modVersion,scnGroup,scnYear
0,2,AllPurp,All,Dy,A,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,39,v900,Base,2019
1,2,AllPurp,All,Dy,P,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,145,v900,Base,2019
2,2,AllPurp,All,Ok,A,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,26,v900,Base,2019
3,2,AllPurp,All,Ok,P,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,92,v900,Base,2019
4,2,AllPurp,All,Pk,A,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,13,v900,Base,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29008935,3629,HBW,sr3p,Dy,P,0.0797,0.0812,0.0,0.0,0.0,...,0.0,0.0,0.0797,0.0,0.0,0.0,25,v900,Needs MAG,2050
29008936,3629,HBW,sr3p,Ok,A,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0,v900,Needs MAG,2050
29008937,3629,HBW,sr3p,Ok,P,0.0455,0.0459,0.0,0.0,0.0,...,0.0,0.0,0.0455,0.0,0.0,0.0,5,v900,Needs MAG,2050
29008938,3629,HBW,sr3p,Pk,A,0.0000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0,v900,Needs MAG,2050


In [ ]:
## replace zeros with NaN
#import numpy as np
#dfCombined_melt_withkeys_pivot_atts.replace(0, np.nan, inplace=True)
#display(dfCombined_melt_withkeys_pivot_atts)

In [107]:
import json

# PREPARE JSON FOR EACH SCENARIO
# scenarioDetails
# attributes
#   aCode
#   aName
#   filterGroup
# filterGroups
#   possible filterGroup Options
#   for each filterGroup possibility, the data for that selection

_df0 = dfCombined_melt_withkeys_pivot_atts.copy()

# Loop through scenarios in dfScn
for _, row in dfScn.iterrows():

    _scenarioCode = row['modVersion'] + '__' + row['scnGroup'] + '__' + str(row['scnYear'])
    print(_scenarioCode)

    # format scenario details for json export
    jsonScenarioDetails = row.to_dict()

    #print(row['modVersion'] + row['scnGroup'] + str(row['scnYear']))
    # Filter the main dataframe based on the scenario
    _df0_filtered = _df0[
        (_df0['modVersion'] == row['modVersion']) & 
        (_df0['scnGroup'] == row['scnGroup']) & 
        (_df0['scnYear'] == row['scnYear'])
    ].copy()

    _df = _df0_filtered.copy()
    #display(_df)

    # add options to _dfFiltersWithOptions
    
    _dfFiltersWithOptions = dfFilters.copy()

    # For each fCode, get the unique values from dfData, and join to dfFilters
    formatted_lists = []
    for fCode in dfFilters['fCode']:
        unique_values = [x for x in _df[fCode].unique() if pd.notna(x)]
        formatted_lists.append(unique_values)
    #display (formatted_lists)

    _dfFiltersWithOptions['fOptions'] = formatted_lists    
    #display(_dfFiltersWithOptions)

    # get list of fitler groups (any column that begins with f)
    fCols = [col for col in dfCombined_melt_withkeys_pivot_atts.columns if col.startswith('f')]
    _df[fCols] = _df[fCols].fillna('')
    _dfFilters = _df[fCols].drop_duplicates()

    # Drop rows with empty strings in ALL columns
    #_dfFilters = _dfFilters[~(_dfFilters == '').all(axis=1)]
    #display(_dfFilters)

    # Convert the values to binary form, so 1 where there is a value and 0 elsewhere
    binary_dfFilters = _dfFilters.applymap(lambda x: 0 if x=='' else 1)
    #display(binary_dfFilters)

    # Get unique rows of combinations
    unique_combinations = binary_dfFilters.drop_duplicates()

    # Convert the unique combinations to a list of tuples
    lstCombinations = list(unique_combinations.itertuples(index=False, name=None))
    #display(lstCombinations)

    # Define a function to convert to binary
    def is_empty_or_not(value):
        return 0 if value=='' else 1

    _dfFilterACodes = pd.DataFrame()

    jsonDataCombined = {}

    # loop through all combinations in list
    for combination in lstCombinations:

        # get list of column names with a 1 for given combination
        _dfFilterColsWithValue1 = [col for idx, col in enumerate(fCols) if combination[idx] == 1]
        #display(_dfFilterColsWithValue1)

        # create string of column names
        _strFilterGroup = '_'.join(_dfFilterColsWithValue1)
        #display(_strFilterGroup)
        
        # Initialize the condition as True for all rows
        condition = pd.Series([True] * len(_df), index=_df.index)

        # Apply the condition programmatically for each column in the list
        for col, comb_value in zip(fCols, combination):
            condition &= _df[col].apply(is_empty_or_not) == comb_value

        _filtered_df = _df[condition].copy()
        _filtered_df.drop(columns=['modVersion','scnGroup','scnYear'], inplace=True)
        #display(_filtered_df)

        # Remove columns where all values are NaN
        _filtered_df_noNA = _filtered_df.dropna(axis=1, how='all')
        #display(_filtered_df_noNA)

        _fColsToDrop = [col for col in _filtered_df_noNA.columns if col.startswith('f') and (_filtered_df_noNA[col] == '').all()]

        # Drop the identified columns - only have fCols with values
        _df2 = _filtered_df_noNA.drop(columns=_fColsToDrop)
        #display(_df2)

        # Get list of columns that begin with "a", create a dataframe the filter group for all possible combinations
        aCols = [col for col in _filtered_df_noNA.columns if col.startswith('a')]
        new_data = pd.DataFrame([{"filterGroup": _strFilterGroup, 'aCode': aCols}])
        new_data = new_data.explode('aCode').reset_index(drop=True)
        _dfFilterACodes = pd.concat([_dfFilterACodes, new_data], ignore_index=True)

        # get unique filter values
        _fColsRemain = [col for col in _df2.columns if col.startswith('f')]
        #display(_fColsRemain)
        if _fColsRemain:
            _dfFilterOptions = _df2[_fColsRemain].drop_duplicates()
        else:
            # add dummy row for no filter
            _dfFilterOptions = pd.DataFrame({'nofilter': ['']})

        #display(_dfFilterOptions)

        # Handle no filter data

        first_row = True

        for index, row in _dfFilterOptions.iterrows():

            _filterRowIndex = '_'.join(row.astype(str).values)

            print(_filterRowIndex)
            
            final_condition = ""

            if _fColsRemain != []:
                # Dynamically construct the filter condition
                shared_columns = set(_dfFilterOptions.columns) & set(_df2.columns)
                conditions = (_df2[col].isin([row[col]]) for col in shared_columns)
                final_condition = next(conditions)

                for condition in conditions:
                    final_condition &= condition

                filtered_df2 = _df2[final_condition].copy()
                filtered_df2.drop(columns=_fColsRemain, inplace=True)
            else: # no filter condition
                filtered_df2 =  _df2

            _df2noI = filtered_df2.reset_index()

            jsonDataForFilterOption = {}

            for index, row in _df2noI.iterrows():
                # Initialize an empty dictionary for the row
                row_dict = {}

                # Loop through each column that starts with 'a'
                for col in _df2noI.columns[_df2noI.columns.str.startswith('a')]:
                    # Add to row_dict if value is greater than 0
                    if row[col] > 0:
                        row_dict[col] = row[col]
                
                # If row_dict is not empty, add it to jsonDataCombined with TAZID as the key
                if row_dict:
                    jsonDataForFilterOption[int(row['TAZID'])] = row_dict  # Cast TAZID to int
            
            # Encapsulate jsonDataCombined under the _filterRowIndex
            final_json = {_filterRowIndex: jsonDataForFilterOption}

            ## Convert to a JSON formatted string
            #json_output = json.dumps(final_json)

            # Append the JSON structure from this iteration to the combined
            jsonDataCombined = {**jsonDataCombined, **final_json}

    _dfAttributesWFilter = pd.DataFrame.merge(dfAttributes, _dfFilterACodes, on='aCode')
    jsonAttributes = _dfAttributesWFilter.to_dict(orient='records')
    jsonFilters = _dfFiltersWithOptions.to_dict(orient='records')

    # Construct final JSON structure
    jsonScenario = {
        'scenarioDetails': jsonScenarioDetails,
        'attributes': jsonAttributes,
        'filters': jsonFilters,
        'data': jsonDataCombined#,
        #'aggregations': aggregations_list
    }

    # Specify the desired file name
    filename = '_site/data/scnData/' + _scenarioCode + outputJsonName

    # Open the file in write mode
    with open(filename, 'w') as file:
        json.dump(jsonScenario, file)

v900__Base__2019
"filterSelection": {"fPurp":"AllPurp","fMode":"All","fTod":"Dy","fPA":"A"}
"filterSelection": {"fPurp":"AllPurp","fMode":"All","fTod":"Dy","fPA":"P"}
"filterSelection": {"fPurp":"AllPurp","fMode":"All","fTod":"Ok","fPA":"A"}
"filterSelection": {"fPurp":"AllPurp","fMode":"All","fTod":"Ok","fPA":"P"}
"filterSelection": {"fPurp":"AllPurp","fMode":"All","fTod":"Pk","fPA":"A"}
"filterSelection": {"fPurp":"AllPurp","fMode":"All","fTod":"Pk","fPA":"P"}
"filterSelection": {"fPurp":"AllPurp","fMode":"Aut","fTod":"Dy","fPA":"A"}
"filterSelection": {"fPurp":"AllPurp","fMode":"Aut","fTod":"Dy","fPA":"P"}
"filterSelection": {"fPurp":"AllPurp","fMode":"Aut","fTod":"Ok","fPA":"A"}
"filterSelection": {"fPurp":"AllPurp","fMode":"Aut","fTod":"Ok","fPA":"P"}
"filterSelection": {"fPurp":"AllPurp","fMode":"Aut","fTod":"Pk","fPA":"A"}
"filterSelection": {"fPurp":"AllPurp","fMode":"Aut","fTod":"Pk","fPA":"P"}
"filterSelection": {"fPurp":"AllPurp","fMode":"Bik","fTod":"Dy","fPA":"A"}
"filterS